In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.notebook import trange, tqdm, tqdm_notebook
from joblib import Parallel, delayed
import re


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\PLA\HCC827-derived OCT mouse'


# Get info

Here we look at stitched images in all z stacks

In [4]:
markers_map = {
    'cycle1': {
        1: 'Hoeschst', 
        4: 'TEAD1 & YAP1'
    },
    'cycle2': {
        1: 'Hoeschst', 
        4: 'CylinE & CDK2'
    },
    'cycle3': {
        1: 'Hoeschst', 
        4: 'P-ERK & c-MYC'
    },
    'cycle4': {
        1: 'Hoeschst', 
        4: 'p-AKT & mTOR'
    },
    'cycle5': {
        1: 'Hoeschst', 
        4: 'Mcl-1 & BAK'
    },
    'cycle6': {
        1: 'Hoeschst',
        2: 'p-EGFR',
        3: 'Tom20',
        4: 'Ki67'
    },
    'cycle7': {
        1: 'Hoeschst',
        2: 'Pan-cytokeratin',
        3: 'Golph4',
        4: 'Bim'
    },
    'cycle8': {
        1: 'Hoeschst',
        2: 'Concanavalin A',
        3: 'Phalloidin',
        4: 'WGA'
    },
    'cycle9': {
        1: 'Hoeschst',
        2: 'NBD-C6'
    },
}

def get_info(data_raw, marker_dict = markers_map):
    timepoints = []
    resolutions = []
    fovs = []
    cycles = []
    afs = []
    channels = []
    markers = []
    rois = []
    z_stacks = []
    paths = [] 
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name and "stitched" not in name \
            and 'Overlay' not in name \
            and 'Composite' not in name \
            and 'defocused' not in dirpath:
                # Get information from image name
                d_split = dirpath.split('\\')
                n_split = name.split('_')
                                
                time = d_split[-1].split('_')[0]
                fov = d_split[-1].split('_')[-1]
                
                try:
                    if 'FW' not in fov:
                        res = '20X'
                        fov = ''
                        ch = int(n_split[2][2])
                        roi = int(n_split[1])
                        z = 1
                    else:
                        res = '40X'
                        ch = int(n_split[3][2])
                        roi = int(n_split[1])
                        z = int(n_split[2][1:])

                    cycle = d_split[-1].split('_')[1]
                    if 'Af' in cycle:
                        after_bleach = True
                        cycle = cycle[2:]
                    else:
                        after_bleach = False
                    marker = marker_dict[cycle][ch]
                except:
                    continue
        
                timepoints.append(time)
                resolutions.append(res)
                fovs.append(fov)
                cycles.append(cycle)
                afs.append(after_bleach)
                channels.append(ch)
                markers.append(marker)
                rois.append(roi)
                z_stacks.append(z)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Timepoint": timepoints,
            "Resolution": resolutions,
            "FOV": fovs,
            "Cycle": cycles,
            "AfBleach": afs,
            "Channels": channels,
            "Markers": markers,
            "ROI": rois,
            "Z": z_stacks,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

In [5]:
df_meta_path = data_dir / 'OCT mouse' / 'ROI' / 'metadata' / 'info.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Folder is already there
Loaded df


In [6]:
df

,Timepoint,Resolution,FOV,Cycle,AfBleach,Channels,Markers,ROI,Z,Path
0,1M,20X,NaN,cycle1,True,1,Hoeschst,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
1,1M,20X,NaN,cycle1,True,4,TEAD1 & YAP1,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
2,1M,20X,NaN,cycle1,True,1,Hoeschst,2,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
3,1M,20X,NaN,cycle1,True,4,TEAD1 & YAP1,2,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
4,1M,20X,NaN,cycle1,True,1,Hoeschst,3,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
...,...,...,...,...,...,...,...,...,...,...
347915,1W,40X,FW3,cycle9,False,2,NBD-C6,40,24,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
347916,1W,40X,FW3,cycle9,False,1,Hoeschst,40,25,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
347917,1W,40X,FW3,cycle9,False,2,NBD-C6,40,25,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."
347918,1W,40X,FW3,cycle9,False,1,Hoeschst,40,26,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\P..."


# Convert data to hdf5 

Convert stitch data to hdf5 format.

For each file we are organized into the format of: File with keys cycles

Attributes are Channels and Markers

In [7]:
import h5py
import multiprocessing
import functools 

def save_hdf5(
    path: str, name: str, data: np.ndarray, attr_dict=None, mode: str = "a"
) -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        chunk_shape = (1,) + data_shape[1:]
        max_shape = (data_shape[0],) + data_shape[1:]
        dset = hf.create_dataset(
            name,
            shape=data_shape,
            maxshape=max_shape,
            chunks=chunk_shape,
            dtype=data_type,
            compression="gzip",
        )
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f"Dataset {name} exists")

    hf.close()

def read_img(path):
    return skimage.io.imread(path)

def joblib_loop(task, pics):
    return Parallel(n_jobs=multiprocessing.cpu_count())(delayed(task)(i) for i in pics)

def save_imgs(df_channel, n, file_path):
    marker = df_channel.iloc[0].Markers
    paths = df_channel.Path.to_numpy()

    imgs = joblib_loop(read_img, paths)
    imgs = np.array(imgs)
    info = {"Cycle": n[0], "Channel": n[1], "Marker": marker, "Z": df_channel.Z.to_numpy()}

    # hdf5 as Channel -> Z mapping
    save_hdf5(file_path, '_'.join(np.array(n).astype(str)), imgs, info)

In [ ]:
df_imgs_path = data_dir / 'OCT mouse' / 'ROI' / 'metadata' / 'imgs.csv'

try:
    df_imgs_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")
    
temp_path = data_dir / 'OCT mouse' / 'ROI' / 'hdf5' / 'raw'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    df_z = df[df.Resolution == '40X']
    df_z = df_z[df_z.AfBleach == False]
    group = df_z.groupby(['Timepoint', 'Resolution', 'FOV', 'AfBleach', 'ROI']) 
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        print(name)
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])

        group_channel = df_group.groupby(['Cycle', 'Channels'])
        for n, df_channel in group_channel:
            marker = df_channel.iloc[0].Markers
            paths = df_channel.Path.to_numpy()

            imgs = joblib_loop(read_img, paths)
            imgs = np.array(imgs)
            info = {"Cycle": n[0], "Channel": n[1], "Marker": marker, "Z": df_channel.Z.to_numpy()}
            
            # hdf5 as Channel -> Z mapping
            save_hdf5(file_path, '_'.join(np.array(n).astype(str)), imgs, info)
    df_imgs = pd.DataFrame(rows, columns=['Timepoint', 'Resolution', 'FOV', 'AfBleach', 'ROI', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Folder is already there
Folder is already there
Created df


  0%|          | 0/306 [00:00<?, ?it/s]

('1M', '40X', 'FW1', False, 1)
('1M', '40X', 'FW1', False, 2)
('1M', '40X', 'FW1', False, 3)
('1M', '40X', 'FW1', False, 4)
('1M', '40X', 'FW1', False, 5)
('1M', '40X', 'FW1', False, 6)
('1M', '40X', 'FW1', False, 7)
('1M', '40X', 'FW1', False, 8)
('1M', '40X', 'FW1', False, 9)
('1M', '40X', 'FW1', False, 10)
('1M', '40X', 'FW1', False, 11)
('1M', '40X', 'FW1', False, 12)
('1M', '40X', 'FW1', False, 13)
('1M', '40X', 'FW1', False, 14)
('1M', '40X', 'FW1', False, 15)
('1M', '40X', 'FW1', False, 16)
('1M', '40X', 'FW1', False, 17)
('1M', '40X', 'FW1', False, 18)
('1M', '40X', 'FW1', False, 19)
('1M', '40X', 'FW1', False, 20)
('1M', '40X', 'FW1', False, 21)
('1M', '40X', 'FW1', False, 22)
('1M', '40X', 'FW1', False, 23)
('1M', '40X', 'FW1', False, 24)
('1M', '40X', 'FW1', False, 25)
('1M', '40X', 'FW1', False, 26)
('1M', '40X', 'FW1', False, 27)
('1M', '40X', 'FW1', False, 28)
('1M', '40X', 'FW1', False, 29)
('1M', '40X', 'FW1', False, 30)
('1M', '40X', 'FW1', False, 31)
('1M', '40X', 'FW

# Get best focus

In [20]:
def detect_blur_fft(image, size=60):
    # grab the dimensions of the image and use the dimensions to
    # derive the center (x, y)-coordinates
    (h, w) = image.shape
    (cX, cY) = (int(w / 2.0), int(h / 2.0))
    
    # compute the FFT to find the frequency transform, then shift
    # the zero frequency component (i.e., DC component located at
    # the top-left corner) to the center where it will be more
    # easy to analyze
    fft = np.fft.fft2(image)
    fftShift = np.fft.fftshift(fft)
    
    # zero-out the center of the FFT shift (i.e., remove low
    # frequencies), apply the inverse shift such that the DC
    # component once again becomes the top-left, and then apply
    # the inverse FFT
    fftShift[cY - size:cY + size, cX - size:cX + size] = 0
    fftShift = np.fft.ifftshift(fftShift)
    recon = np.fft.ifft2(fftShift)
    
    magnitude = 20 * np.log(np.abs(recon))
    mean = np.mean(magnitude)
    
    return mean

In [21]:
df_imgs.columns

Index(['Timepoint', 'Resolution', 'FOV', 'AfBleach', 'ROI', 'Path'], dtype='object')

In [22]:
csv_blur = data_dir / 'OCT mouse' / 'ROI'/ 'metadata' / "blur.csv"

try:
    csv_blur.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

k = '1'    

if not csv_blur.is_file():
    max_blur_values = []
    for row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
        path = row.Path
        
        with h5py.File(path, "r") as f:
            for k in f.keys():
                if f[k].attrs['Channel'] == 1:
                    cycle = k.split('_')[0]
                    channel = f[k].attrs['Channel']
                    imgs = f[k]
                    blur_scores = joblib_loop(detect_blur_fft, imgs)
                    blur_max = np.argmax(blur_scores)
                    values = list(row[1:])
                    values.append(cycle)
                    values.append(blur_max)
                    max_blur_values.append(values)
    df_blur_max = pd.DataFrame(max_blur_values, columns=['Timepoint', 'Resolution', 'FOV', 'AfBleach', 'ROI', 'Path', 'Cycle', 'Z_stack_focus'])
    df_blur_max.to_csv(csv_blur, index=False)
else:
    df_blur_max = pd.read_csv(csv_blur)

Folder is already there


  0%|          | 0/306 [00:00<?, ?it/s]

In [23]:
df_blur_max

,Timepoint,Resolution,FOV,AfBleach,ROI,Path,Cycle,Z_stack_focus
0,1M,40X,FW1,False,1,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle1,11
1,1M,40X,FW1,False,1,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle2,15
2,1M,40X,FW1,False,1,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle3,14
3,1M,40X,FW1,False,1,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle4,15
4,1M,40X,FW1,False,1,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle5,11
...,...,...,...,...,...,...,...,...
2650,1W,40X,FW3,False,43,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle2,15
2651,1W,40X,FW3,False,44,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle1,18
2652,1W,40X,FW3,False,44,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle2,16
2653,1W,40X,FW3,False,45,Y:\coskun-lab\Thomas\15_PLA\data\OCT mouse\ROI...,cycle1,13


# Get shift per cycle

In [ ]:
from skimage.registration import phase_cross_correlation
from IPython.display import display, clear_output

def get_shift_between_cycle(imgs, cycles):
    """Function to get shift within each cycle for the DAPI channel

    Args:
        df (pd DataFrame) : info dataframe for images in dapi channel accross cycle

    Returns:
        shift dictionnary
    """
    shift_dict = {}
    
    # Max shift accross all cycle:
    max_shift_x = 0
    min_shift_x = 0
    max_shift_y = 0
    min_shift_y = 0
    reference_dapi = imgs[0]
    cycles_it = cycles[1:]
    for i, img_dapi in enumerate(imgs[1:]):
        
        # print(f'Registration cycle {cycles[i+1]}', flush=True)
        # Get image shift y and x and save to shift_dict
        try:
            shift, _, _ = phase_cross_correlation(
                reference_dapi, img_dapi, 
                upsample_factor=1
            )  # Shift vector required to register moving images with reference images. Axis orderingis constitent with Y,X
        except ValueError as err:
            print(err)
        shift_y, shift_x = shift[0], shift[1]
        shift_dict[cycles[i+1]] = {"shift_x": shift_x, "shift_y": shift_y}

        # Update max shift
        max_shift_x = shift_x if shift_x > max_shift_x else max_shift_x
        min_shift_x = shift_x if shift_x < min_shift_x else min_shift_x
        max_shift_y = shift_y if shift_y > max_shift_y else max_shift_y
        min_shift_y = shift_y if shift_y < min_shift_y else min_shift_y

    max_shift_x = int(max_shift_x)
    min_shift_x = int(min_shift_x)
    max_shift_y = int(max_shift_y)
    min_shift_y = int(min_shift_y)

    shift_dict["max"] = {
        "shift_x": max_shift_x,
        "shift_y": max_shift_y,
    }
    shift_dict["min"] = {
        "shift_x": min_shift_x,
        "shift_y": min_shift_y,
    }

    return shift_dict

In [ ]:
n_cycle = len(df_blur_max.Cycle.unique())

In [ ]:
shift_csv = data_dir / 'OCT mouse' /  'ROI'/  'metadata' / "shift.csv"

try:
    shift_csv.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

k = '1'    

if not shift_csv.is_file():
    dfs_shift = []
    for row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
        df_query = df_blur_max.query('Timepoint==@row[1] & FOV==@row[3] & ROI==@row[5]')
        if len(df_query) < n_cycle:
            continue 
        path = row.Path
        
        imgs_dapi = []
        cycles = []
        with h5py.File(path, "r") as f:
            for k in f.keys():
                if f[k].attrs['Channel'] == 1:
                    df_c = df_query[df_query['Cycle'] == f[k].attrs['Cycle']]
                    best_focus = df_c.Z_stack_focus.item()
                    imgs_dapi.append(f[k][best_focus, ...])
                    cycles.append(int(k.split('_')[0][-1]))
                    
        shift = get_shift_between_cycle(imgs_dapi, cycles)
        df_temp = pd.DataFrame(shift)
        old_idx = df_temp.index.to_frame() # Convert index to dataframe
        old_idx.insert(0, 'ROI', row.ROI) # Insert new level at specified location
        old_idx.insert(0, 'FOV', row.FOV)
        old_idx.insert(0, 'Timepoint', row.Timepoint)
        df_temp.index = pd.MultiIndex.from_frame(old_idx) # Convert back to MultiIndex
        dfs_shift.append(df_temp)
    
    df_shift = pd.concat(dfs_shift)
    df_shift.to_csv(shift_csv , index=True)
else:
    df_shift = pd.read_csv(shift_csv)

In [ ]:
df_shift.set_index(['Timepoint', 'FOV', 'ROI', 'Unnamed: 3'], inplace=True)
df_shift.head()

# Get shift cropped images

In [ ]:
from skimage import transform
from functools import partial

def crop_images(imgs, df_shift, cycles, upscale=1):
    # Get max shift values
    max_shift_x = df_shift.loc['shift_x', 'max']
    min_shift_x = df_shift.loc['shift_x', 'min']
    max_shift_y = df_shift.loc['shift_y', 'max']
    min_shift_y = df_shift.loc['shift_y', 'min']
    
    imgs_cropped = []
    for idx, img in enumerate(imgs):
        if cycles[idx] == '1':
            # Crop image
            res_cropped = img[:, max_shift_y:min_shift_y-1, max_shift_x:min_shift_x-1]
        else:
            #Apply shift
            shift_x, shift_y = (
                df_shift.loc['shift_x', cycles[idx]],
                df_shift.loc['shift_y', cycles[idx]],
            )
            # rows, cols = img.shape
            # M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
            # res = cv2.warpAffine(img, M, (cols, rows))
            shift_x = int(shift_x*upscale)
            shift_y = int(shift_y*upscale)
            
            t = transform.AffineTransform(translation=[-shift_x, -shift_y])
            f_partial = partial(transform.warp, inverse_map=t,mode='constant', cval=0, preserve_range=True)
            # res = transform.warp(img, t, mode='constant', cval=0, preserve_range=True)
            res = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(f_partial)(i) for i in img)
            res = np.array(res)
            
            # Crop image
            res_cropped = res[:, max_shift_y:min_shift_y-1, max_shift_x:min_shift_x-1]
            res_cropped.astype(img.dtype)
        imgs_cropped.append(res_cropped)
    return imgs_cropped

In [ ]:
# n = 5
# for i, row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
#     path = row.Path
#     df_query = df_blur_max.query('Timepoint==@row[1] & FOV==@row[3] & ROI==@row[5]')
#     imgs = []
#     cycles = []
#     channels = []
#     markers = []
#     with h5py.File(path, "r") as f:
#         for k in f.keys():
#             df_c = df_query[df_query['Cycle'] == f[k].attrs['Cycle']]
#             if len(df_c) == 0:
#                 continue 
#             best_focus = df_c.Z_stack_focus.item()
#             # imgs.append(f[k][best_focus-n:best_focus+n, ...])
#             cycles.append(int(k.split('_')[0][-1]))
#             channels.append(int(k.split('_')[-1]))
#             markers.append( f[k].attrs['Marker'])
#     # df_shift_roi = df_shift.loc[row[1], row[3], row[5]]
#     # imgs_cropped = crop_images(imgs, df_shift_roi, cycles)
#     break

In [ ]:
df_reg_path = data_dir / 'OCT mouse' / 'ROI' / 'metadata' / 'imgs_reg.csv'
n = 5

try:
    df_reg_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")
    
temp_path = data_dir / 'OCT mouse' / 'ROI' / 'hdf5' / 'registered'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_reg_path.is_file()

if not df_exist:
    df_reg = df_imgs.copy()
    for row in tqdm(df_imgs.itertuples(), total=len(df_imgs)):
        # Get Path
        path = row.Path
        file_name = row.Path.split('\\')[-1]
        file_path = temp_path / file_name
        
        # Check if file exist
        if file_path.is_file():
            print(file_name+' exists!')
            df_reg.loc[row.Index, 'Path'] = file_path
            continue
        
        # Query from blur dataframe 
        df_query = df_blur_max.query('Timepoint==@row[1] & FOV==@row[3] & ROI==@row[5]')
        imgs = []
        cycles = []
        channels = []
        markers = []
        with h5py.File(path, "r") as f:
            for k in f.keys():
                df_c = df_query[df_query['Cycle'] == f[k].attrs['Cycle']]
                if len(df_c) == 0:
                    continue 
                best_focus = df_c.Z_stack_focus.item()
                if best_focus < n:
                    best_focus = n
                elif best_focus > len(f[k])-n:
                    best_focus = len(f[k])-n
                imgs.append(f[k][best_focus-n:best_focus+n, ...])
                cycles.append(int(k.split('_')[0][-1]))
                channels.append(int(k.split('_')[-1]))
                markers.append( f[k].attrs['Marker'])
        
        try:
            df_shift_roi = df_shift.loc[row[1], row[3], row[5]]
            imgs_cropped = crop_images(imgs, df_shift_roi, np.array(cycles).astype(str))
        except Exception as e:
            print(row, str(e))
            continue
        info = {"Cycle": cycles, "Channel": channels, "Marker": markers}
        
        try:
            imgs_stacked = np.stack(imgs_cropped)
        except:
            min_z = imgs_cropped[0].shape[0]
            for i in imgs_cropped:
                min_z = i.shape[0] if i.shape[0] < min_z else min_z
            imgs_stacked = np.stack([i[:min_z,...] for i in imgs_cropped])
        # hdf5 as Channel -> Z mapping
        save_hdf5(file_path, 'imgs', imgs_stacked, info)
        df_reg.loc[row.Index, 'Path'] = file_path
        
    df_reg.to_csv(df_reg_path, index=False)
else:
    print('Loaded df')
    df_reg = pd.read_csv(df_reg_path)